In [1]:
# 2019 Q4
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/SOTF_vs_Legacy_performance'

In [2]:
list_sotf=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/SOTF_vs_Legacy_performance/BL Store List Report from Dom 12.20.19.xlsx",
                       dtype=str,usecols=['Store','SOTF'])
list_sotf=list_sotf[list_sotf['SOTF']=="Yes"]
list_sotf=list_sotf[list_sotf['Store'].apply(lambda x: x.isdigit())]['Store'].unique().tolist()
len(list_sotf)

479

In [3]:
store_list_files=glob.glob("/home/jian/BigLots/static_files/Store_list/MediaStormStores20*.txt")
store_list_files=sorted(store_list_files,reverse=True)
latest_store_list=store_list_files[0]
store_list_files.remove(latest_store_list)

print(latest_store_list)
store_list_files

/home/jian/BigLots/static_files/Store_list/MediaStormStores20200201-134613-934.txt


['/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191201-133313-746.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191101-134011-956.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190801-135940-752.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190401-134939-117.txt',
 '/home/jian/BigLots/static_files/Store_list/MediaStormStores20190301-134800-131.txt',
 '/home/jian/BigLots/static_files/Store_lis

In [4]:
weeks_2019Q4=[datetime.date(2019,11,9)+datetime.timedelta(days=x*7) for x in range(13)]

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
folder_2019_by_week="/home/jian/BigLots/"

Q4_files_2019_all_types=list(recursive_file_gen(folder_2019_by_week))
Q4_files_2019_all_types=[x for x in Q4_files_2019_all_types if "aily" in x]
Q4_files_2019=[]
for week_end_date in weeks_2019Q4:
    file_path=[x for x in Q4_files_2019_all_types if str(week_end_date) in x]
    Q4_files_2019=Q4_files_2019+file_path
    
print(len(Q4_files_2019))
Q4_files_2019=sorted(Q4_files_2019)
Q4_files_2019

13


['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-16/MediaStormDailySales20191119-112232-478.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-23/MediaStormDailySales20191126-112901-552.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-30/MediaStormDailySales20191203.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-07/MediaStormDailySales20191211.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-21/MediaStormDailySales20191226-122746-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-28/MediaStormDailySales20191231-112945-515.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-04/MediaStormDailySales20200107-112859-015.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-11/MediaStormDailySales20200114-115009-14

In [5]:
def week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=(5-x.weekday()))
    return y


In [6]:
def load_and_agg_df(file_list):
    
    i_counter=0

    sales_agg_df=pd.DataFrame()
    ids_by_week_store=pd.DataFrame()

    for file_path in file_list:
        df=pd.read_table(file_path,dtype=str,sep="|",usecols=None,nrows=None)
        df=df.drop_duplicates()            

        df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())
        df['item_transaction_amt']=df['item_transaction_amt'].astype(float)

        df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")

        date_max=df['transaction_dt'].max()
        date_min=df['transaction_dt'].min()

        # print(i_counter,date_max,date_min,datetime.datetime.now())

        if ((date_max-date_min).days==6) & (date_max.weekday()==5):
            df['week_end_dt']=date_max
            df_agg_sales=df.groupby(['location_id','week_end_dt','rewards_label'])['item_transaction_amt'].sum().to_frame().reset_index()
            df_agg_sales=df_agg_sales.rename(columns={"item_transaction_amt":"sales"})

            df_agg_trans=df[['location_id','transaction_dt','week_end_dt','transaction_id','customer_id_hashed','rewards_label']].drop_duplicates()
            df_agg_trans['transactions']=1
            df_agg_trans=df_agg_trans.groupby(['location_id','week_end_dt','rewards_label'])['transactions'].sum().to_frame().reset_index()

            df_agg_sales=pd.merge(df_agg_sales,df_agg_trans,on=['location_id','week_end_dt','rewards_label'],how="outer")

            df=df[df['rewards_label']=="Rewards"]
            df=df[['location_id','week_end_dt','customer_id_hashed']].drop_duplicates()

        else:
            print("Date in the data not 7 days",file_path)
            df=pd.DataFrame()
            df_agg_sales=pd.DataFrame()

        sales_agg_df=sales_agg_df.append(df_agg_sales)
        ids_by_week_store=ids_by_week_store.append(df)
        i_counter+=1


    return sales_agg_df,ids_by_week_store

In [7]:
sales_agg_2019, ids_by_week_store_2019 = load_and_agg_df(Q4_files_2019)

In [8]:
list_new_sing_up_file=list(recursive_file_gen("/home/jian/BigLots/"))
list_new_sing_up_file=[x for x in list_new_sing_up_file if "MasterWeekly" in x]
list_new_sing_up_file=[x for x in list_new_sing_up_file if "/MediaStorm_" in x]
list_new_sing_up_file=[x for x in list_new_sing_up_file if x.split("/MediaStorm_")[1][:10]>="2019-11-03"]
list_new_sing_up_file=[x for x in list_new_sing_up_file if x.split("/MediaStorm_")[1][:10]<="2020-02-01"]
list_new_sing_up_file.sort()
print(len(list_new_sing_up_file))

13


In [9]:
df_new_sign_ups=pd.DataFrame()
for file in list_new_sing_up_file:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed'])
    df_new_sign_ups=df_new_sign_ups.append(df)
df_new_sign_ups['id_new_sign_label']="new_sign_ups"

df_new_sign_ups=df_new_sign_ups.drop_duplicates()
df_new_sign_ups.shape,df_new_sign_ups['customer_id_hashed'].nunique()

((1837901, 2), 1837901)

In [10]:
ids_by_week_store_2019=pd.merge(ids_by_week_store_2019,df_new_sign_ups,on="customer_id_hashed",how="left")
ids_by_week_store_2019['id_new_sign_label']=ids_by_week_store_2019['id_new_sign_label'].fillna("not_new_signed")

In [61]:
ids_by_week_store_2019['customer_id_hashed'].nunique()

10401633

In [26]:
ids_by_week_store_2019['Store_Type']=np.where(ids_by_week_store_2019['location_id']=="6990","Online",
                                    np.where(ids_by_week_store_2019['location_id'].isin(list_sotf),"SOTF","Legacy")                                        
                                    )
ids_by_week_store_2019['Store_Type'].unique().tolist()

['Legacy', 'SOTF', 'Online']

In [29]:
df_store_count=ids_by_week_store_2019.groupby("Store_Type")['location_id'].nunique().to_frame().reset_index()
df_store_count['location_id'].sum()

1423

In [32]:
sales_agg_2019.head(2)

,location_id,week_end_dt,rewards_label,sales,transactions
0,1,2019-11-09,Non_Rewards,28304.05,860
1,1,2019-11-09,Rewards,48509.16,1156


In [33]:
sales_agg_2019['sales'].sum()

1604694585.9500022

In [35]:
1604694585.9500022/10**9

1.6046945859500021

In [36]:
print(ids_by_week_store_2019['customer_id_hashed'].nunique())

df_unique_shoppers_per_type=ids_by_week_store_2019.groupby("Store_Type")['customer_id_hashed'].nunique().to_frame().reset_index()
df_unique_shoppers_per_type

10401633


,Store_Type,customer_id_hashed
0,Legacy,7019354
1,Online,60825
2,SOTF,3656217


In [42]:
df_unique_shoppers_per_type_2=ids_by_week_store_2019.groupby(["Store_Type",'id_new_sign_label'])['customer_id_hashed'].nunique().to_frame().reset_index()
df_unique_shoppers_per_type_2=df_unique_shoppers_per_type_2.pivot_table(index="Store_Type",columns="id_new_sign_label",values="customer_id_hashed").reset_index()
df_unique_shoppers_per_type_2=df_unique_shoppers_per_type_2.rename(columns={"new_sign_ups":"new_shopper_count","not_new_signed":"existing_shopper_count"})
df_unique_shoppers_per_type=pd.merge(df_unique_shoppers_per_type,df_unique_shoppers_per_type_2,on="Store_Type")
df_unique_shoppers_per_type

,Store_Type,customer_id_hashed,new_shopper_count,existing_shopper_count
0,Legacy,7019354,761814,6257540
1,Online,60825,10492,50333
2,SOTF,3656217,409979,3246238


In [39]:
sales_agg_2019['Store_Type']=np.where(sales_agg_2019['location_id']=="6990","Online",
                                    np.where(sales_agg_2019['location_id'].isin(list_sotf),"SOTF","Legacy")                                     
                                    )

sales_agg_2019.head(5)

,location_id,week_end_dt,rewards_label,sales,transactions,Store_Type
0,1,2019-11-09,Non_Rewards,28304.05,860,SOTF
1,1,2019-11-09,Rewards,48509.16,1156,SOTF
2,1001,2019-11-09,Non_Rewards,22685.93,724,Legacy
3,1001,2019-11-09,Rewards,28111.85,627,Legacy
4,1003,2019-11-09,Non_Rewards,14093.81,673,Legacy


In [58]:
sales_agg_2019.shape

(36882, 6)

In [52]:
df_output_1_store_count=sales_agg_2019.groupby("Store_Type")['location_id'].nunique().to_frame().reset_index()
df_output_2_sales_rewards=sales_agg_2019[sales_agg_2019['rewards_label']=="Rewards"].groupby("Store_Type")['sales','transactions'].sum().reset_index()
df_output_2_sales_rewards=df_output_2_sales_rewards.rename(columns={"sales":"rewards_sales","transactions":"rewards_trans"})
df_output_2_sales_nonrewards=sales_agg_2019[sales_agg_2019['rewards_label']=="Non_Rewards"].groupby("Store_Type")['sales','transactions'].sum().reset_index()
df_output_2_sales_nonrewards=df_output_2_sales_nonrewards.rename(columns={"sales":"Nonrewards_sales","transactions":"Nonrewards_trans"})
df_output_2_sales=pd.merge(df_output_2_sales_rewards,df_output_2_sales_nonrewards,on="Store_Type",how="outer")

In [56]:
df_output_final=pd.merge(df_output_1_store_count,df_output_2_sales,on="Store_Type",how="outer")
df_output_final=pd.merge(df_output_final,df_unique_shoppers_per_type,on="Store_Type",how="outer")
df_output_final

,Store_Type,location_id,rewards_sales,rewards_trans,Nonrewards_sales,Nonrewards_trans,customer_id_hashed,new_shopper_count,existing_shopper_count
0,Legacy,958,6.598408e+08,16905174,3.696799e+08,13750826,7019354,761814,6257540
1,Online,1,7.152012e+06,83006,5.488623e+06,65095,60825,10492,50333
2,SOTF,464,3.597390e+08,8392235,2.027942e+08,6636078,3656217,409979,3246238


In [57]:
df_output_final.insert(df_output_final.columns.tolist().index("location_id")+1,"total_sales",df_output_final['rewards_sales']+df_output_final['Nonrewards_sales'])
df_output_final.insert(df_output_final.columns.tolist().index("total_sales")+1,"total_trans",df_output_final['rewards_trans']+df_output_final['Nonrewards_trans'])
df_output_final=df_output_final.rename(columns={"location_id":"store_count_Q4","customer_id_hashed":"total_shoppers"})
df_output_final

,Store_Type,store_count_Q4,total_sales,total_trans,rewards_sales,rewards_trans,Nonrewards_sales,Nonrewards_trans,total_shoppers,new_shopper_count,existing_shopper_count
0,Legacy,958,1.029521e+09,30656000,6.598408e+08,16905174,3.696799e+08,13750826,7019354,761814,6257540
1,Online,1,1.264063e+07,148101,7.152012e+06,83006,5.488623e+06,65095,60825,10492,50333
2,SOTF,464,5.625332e+08,15028313,3.597390e+08,8392235,2.027942e+08,6636078,3656217,409979,3246238


In [59]:
writer=pd.ExcelWriter("./BL_Q4_store_type_store_level_summary_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_final.to_excel(writer,"store_level_summary",index=False)
sales_agg_2019.to_excel(writer,"Sales_data_by_Store",index=False)
writer.save()